"Room" where user can input character name. See bottom code cell for a version with storage. It's not very pretty, but it should work. We might want to make the character length shorter, but we can figure that out later.

attempt 1) can easily get and print out multiple character strings

In [1]:
.ORIG x3000   

NAMING: GETC ;; get input, value gets stored in R0
        PUTC ;; print input

HALT

.END

Assembled! Use %dis or %dump to examine; use %exe to run.


In [2]:
%exe

name
Computation completed
Instructions: 6
Cycles: 54 (0.000027 milliseconds)

Registers:
PC: x048E
N: 0 Z: 0 P: 1 
R0: x006E R1: x0000 R2: x0000 R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x3002 


attempt 2) get, store, and print multiple character string

able to input multiple characters, but only store one

In [134]:
.ORIG x3100   

LEA R1, NAME ;; load R1 with .BLKW

NAMING: GETC ;; get input, value gets stored in R0
        STR R0, R1, #0 ;; load R0 into first .BLKW location
        ADD R1, R1, #1 ;; go to next .BLKW location
        BR FIN

FIN: LEA R0, NAME
     PUTS

HALT

NAME: .BLKW #10

.END

Assembled! Use %dis or %dump to examine; use %exe to run.


In [135]:
%exe

orange
Computation completed
Instructions: 8
Cycles: 70 (0.000035 milliseconds)

Registers:
PC: x048E
N: 0 Z: 0 P: 1 
R0: x006F R1: x3104 R2: x0000 R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x3104 


attempt 3) messing around with getting storage to work

using IN seems to be best, since the user can type the entire name in the first box, have each character stored in the open slots, then hitting return lets remaining slots be left blank

In [4]:
.ORIG x3100   

;;name room

;;prints room description, sets up for running NAMING
NAME_SETUP: LEA R0, ROOM_INFO ;; load R0 w/string stored in ROOM_INFO
            PUTSP ;; print ROOM_INFO
            LEA R1, NAME ;; load R1 with NAME starting location
            ADD R2, R2, #8 ;; set R2 to the number of input characters allowed
            AND R3, R3, #0 ;; set R3 = 0
            STI R3, R1, #9 ;; store R3 in NAME+9

;; asks user to input a character 10 times and stores characters to print later
NAMING: GETC ;; get input, value gets stored in R0
        STR R0, R1, #0 ;; load R0 into first .BLKW location
        ADD R1, R1, #1 ;; go to next .BLKW location
        ADD R2, R2, #-1 ;; decrement R2
        BRp NAMING ;; if R2 is positive, repeat NAMING to get next character
        BRnz FIN ;; if R2 is 0, go to FIN to print out name

FIN: LEA R0, NAME
     PUTS

HALT

NAME: .BLKW #10
ROOM_INFO: .STRINGC "Give your character a name! Type a name for your character in the box, then hit return to finish.\n"

.END

Assembled! Use %dis or %dump to examine; use %exe to run.


In [5]:
%exe

Give your character a name! Type a name for your character in the box, then hit return to finish.
Ruby



Ruby============================================================
Computation completed
Instructions: 3846
Cycles: 25908 (0.012954 milliseconds)

Registers:
PC: x048E
N: 0 Z: 0 P: 1 
R0: x310F R1: x3117 R2: x0000 R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x310F 


In [6]:
%dis

Memory disassembled:
NAME_SETUP: x3100: xE018  LEA R0, ROOM_INFO                         [line: 5]
           x3101: xF024  PUTSP                                     [line: 6]
           x3102: xE20C  LEA R1, NAME                              [line: 7]
           x3103: x14A8  ADD R2, R2, #8                            [line: 8]
           x3104: x56E0  AND R3, R3, #0                            [line: 9]
           x3105: xB649  STI R3, x314F                             [line: 10]
NAMING:    x3106: xF020  GETC                                      [line: 13]
           x3107: x7040  STR R0, R1, 0                             [line: 14]
           x3108: x1261  ADD R1, R1, #1                            [line: 15]
           x3109: x14BF  ADD R2, R2, #-1                           [line: 16]
           x310A: x03FB  BRp NAMING                                [line: 17]
           x310B: x0C00  BRnz FIN (or 0)                           [line: 18]
FIN:       x310C: xE002  LEA R0, NAME          

In [230]:
.ORIG x3100   

;;name room

;;prints room description, sets up for running NAMING
NAME_SETUP: LEA R0, ROOM_INFO ;; load R0 w/string stored in ROOM_INFO
            PUTSP ;; print ROOM_INFO
            LEA R1, NAME ;; load R1 with NAME starting location
            LD R2, MAX_LEN

NAMING: GETC ;; get input, value stored in R0
        STR R0, R1, #0
        ADD R1, R1, #1
        ADD R0, R0, #0
        BRz FIN ;; if character is zero terminator, done
        ADD R2, R2, #-1
        BRz FULL ;; if we're out of space, done
        BRnzp NAMING

FULL: ADD R1, R1, #-1
      AND R2, R2, #0
      STR R2, R1, #0

FIN: LEA R0, NAME
     PUTS

HALT

NAME: .BLKW #10
MAX_LEN: .FILL #10
ROOM_INFO: .STRINGC "Give your character a name! Type a name for your character in the box, then hit return to finish.\n"

.END

Assembled! Use %dis or %dump to examine; use %exe to run.


In [236]:
%exe

Give your character a name! Type a name for your character in the box, then hit return to finish.
Abcdefghij
Abcdefghi============================================================
Computation completed
Instructions: 3948
Cycles: 26685 (0.013343 milliseconds)

Registers:
PC: x048E
N: 0 Z: 0 P: 1 
R0: x3112 R1: x311B R2: x0000 R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x3112 


In [224]:
%step

Stepping...  => read, <= write, (Instructions/Cycles):
    PC <= x310C
(3726/24923) ADD R1, R1, #-1 [18] (x310C*: x127F)
    R1 <= x3111
    NZP <= (0, 0, 1)

Registers:
PC: x310C
N: 0 Z: 0 P: 1 
R0: x0052 R1: x3111 R2: x0009 R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x3105 


In [232]:
%dis

Memory disassembled:
NAME_SETUP: x3100: xE01C  LEA R0, ROOM_INFO                         [line: 5]
           x3101: xF024  PUTSP                                     [line: 6]
           x3102: xE20F  LEA R1, NAME                              [line: 7]
           x3103: x2418  LD R2, MAX_LEN                            [line: 8]
NAMING:    x3104: xF020  GETC                                      [line: 10]
           x3105: x7040  STR R0, R1, 0                             [line: 11]
           x3106: x1261  ADD R1, R1, #1                            [line: 12]
           x3107: x1020  ADD R0, R0, #0                            [line: 13]
           x3108: x0406  BRz FIN (or 6)                            [line: 14]
           x3109: x14BF  ADD R2, R2, #-1                           [line: 15]
           x310A: x0401  BRz FULL (or 1)                           [line: 16]
           x310B: x0FF8  BRnzp NAMING                              [line: 17]
FULL:      x310C: x127F  ADD R1, R1, #-1      